# INFO - BERT embeddings ja relatsioonide treening
EstNLTK 1.6 arendusharust saadud [BertTaggeriga](https://github.com/estnltk/estnltk/tree/devel_1.6/estnltk/taggers/embeddings) sõnadele BERT peidetud kihtide vektorite lisamine. Saadud vektoreid kasutatakse ajaseoseid tuvastavate mudelite treenimiseks. Treenimiseks katsetati erinevaid mudeleid ning nende tulemuste järgi valiti sobivaim, mida üritati veel edasi seadistada.

Katsetamiseks võeti BERT-i nelja viimase kihi vektorid kokku liidetuna, nelja viimase kihi vektorite summa, viimase kihi vektori, eelviimase kihi vektori ja kõikide kihtide vektorite summa. Kuna on sündmusi ja ajaväljendeid,  mis koosnevad  mitmest  sõnast, katsetati neile vektorite  valimisel  erinevaid lähenemisi. Sündmuste puhul prooviti: 
- ainult fraasi peasõna vektorit.
- fraasi sõnade vektorite keskmist väärtust
- fraasi sõnade vektorite kaalutatud keskmist väärtust, kus kõrgem kaal on peasõnal
- fraasi  sõnade  vektorite  kaalutatud  keskmist  väärtust,  kus  kaalud  on  sõnaliikide  järgi (suurimast  kaalust  alates:  üheselt  tegusõnaks  liigitatud  sõnad,  nud-ja  tud-vormid, käändsõnad, ülejäänud sõnaliigid)

Mitmesõnaliste ajaväljendite puhul otsustati kasutada ainult väljendi sõnede vektorite keskmist väärtust.

**NB!** Mugavamaks navigeerimiseks on soovitav kasutada Colab keskkonnas sisukorda või muus keskkonnas sarnast *add-on*'i.

# Vajalike teekide installimine

In [1]:
!pip install transformers
!pip install conllu==3.1.1 estnltk==1.6.7b0

# Drive'st lugemiseks

In [2]:
#Failide lugemiseks Colabis
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/My Drive/Colab Notebooks/")

Mounted at /content/drive


In [3]:
drive_path = "Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/"

# EstNLTK'le EstBERT embeddings lisamine

In [4]:
!cp -R ./Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/embeddings /usr/local/lib/python3.7/dist-packages/estnltk/taggers/

# Imports

In [5]:
from estnltk import Text, Layer
from estnltk.taggers.embeddings.bert.bert_tagger import BertTagger
from estnltk.converters import text_to_json, json_to_text
import json
import numpy as np
import pickle
import random

# Artiklid

In [6]:
def loe_artiklid():
  drive_path = "Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/"
  artiklite_kaust = drive_path + "artiklid"

  laetud_artiklid = []

  with os.scandir(artiklite_kaust) as entries:
    for entry in entries:
      if entry.is_file():
        data = None
        with open(entry, 'r') as fp:
          data = json.load(fp)
        laetud_artiklid.append(json_to_text(data))

  print("Lugesin sisse", len(laetud_artiklid), "artiklit.")
  return laetud_artiklid

#laetud_artiklid = loe_artiklid()

# BertTagger ja embeddingute lisamine

In [7]:
def salvesta_embeddingutega(text_object):
  text_json = text_to_json(text_object)
  with open(drive_path + "artiklid_embeddingutega/" + text_object.meta['filename'] + "_embeddings" + '.json', 'w') as fp:
      json.dump(text_json, fp)
  print("Salvestatud", drive_path + "artiklid_embeddingutega/" + text_object.meta['filename'] + '.json')

In [8]:
def lisa_embeddingud(artiklid):
  bert_tagger = BertTagger(bert_location='./Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/EstBERT', token_level=False)
  bert_tagger_add = BertTagger(bert_location='./Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/EstBERT', token_level=False, method="add", output_layer="bert_embeddings_add")
  bert_tagger_lastlayer = BertTagger(bert_location='./Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/EstBERT', token_level=False, bert_layers=[-1], output_layer="bert_embeddings_lastlayer")
  bert_tagger_penultimate = BertTagger(bert_location='./Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/EstBERT', token_level=False, bert_layers=[-2], output_layer="bert_embeddings_penultimatelayer")
  bert_tagger_all_add = BertTagger(bert_location='./Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/EstBERT', token_level=False, method="add", bert_layers=[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12], output_layer="bert_embeddings_all_add")

  for artikkel in artiklid:
    bert_tagger.tag(artikkel)
    bert_tagger_add.tag(artikkel)
    bert_tagger_lastlayer.tag(artikkel)
    bert_tagger_penultimate.tag(artikkel)
    bert_tagger_all_add.tag(artikkel)

    # Mõned embeddingud on numpy.ndarray'ga. Neid ei saa kergelt json faili salvestada
    eemaldada = None
    for layer in ["bert_embeddings", "bert_embeddings_add", "bert_embeddings_lastlayer", "bert_embeddings_penultimatelayer", "bert_embeddings_all_add"]:
      # Eemalda eelmine kiht
      if eemaldada:
        artikkel.pop_layer(eemaldada)
      uus_kiht = Layer(name=layer + "_fixed", text_object=artikkel, attributes=['token', 'bert_embedding'], ambiguous=True)
      for entry in getattr(artikkel, layer):
        new_token = []
        for tk in entry.token[0]:
          new_token.append(tk)
        embedding_info = entry.bert_embedding
        # Kõik np.ndarray'd tuleb teha listiks
        new_embeddings = []
        for embedding in embedding_info:
          try:
            new_embeddings = embedding.tolist()
          except:
            new_embeddings = embedding
        
        uus_kiht.add_annotation(entry.base_span, token=new_token, bert_embedding=new_embeddings)
      # Lisada kiht
      artikkel.add_layer(uus_kiht)
      eemaldada = layer
    
    artikkel.pop_layer(eemaldada)
    salvesta_embeddingutega(artikkel)

  del bert_tagger
  del bert_tagger_add
  del bert_tagger_lastlayer
  del bert_tagger_penultimate
  del bert_tagger_all_add

#lisa_embeddingud(laetud_artiklid)

In [9]:
def loe_embeddingutega_artiklid():
  drive_path = "Loputoo_Ajaseoste_automaatne_tuvastamine_tekstis/"
  artiklite_kaust = drive_path + "artiklid_embeddingutega"

  laetud_artiklid_embeddingutega = []

  with os.scandir(artiklite_kaust) as entries:
    for entry in entries:
      if entry.is_file():
        data = None
        with open(entry, 'r') as fp:
          data = json.load(fp)
        print(data)
        laetud_artiklid_embeddingutega.append(json_to_text(data))

  print("Lugesin sisse", len(laetud_artiklid_embeddingutega), "artikklit.")
  return laetud_artiklid_embeddingutega

laetud_artiklid_embeddingutega = loe_embeddingutega_artiklid()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Meetodid artiklist sündmuste embeddingute saamiseks

In [11]:
# Sündmuse fraasist peasõna embeddingu võtmine
# ühe sõna embedding'u väljavalimist (valida class märgendiga EVENT)
def main_event_embedding(artikkel, layer_name="bert_embeddings_fixed"):
  embeddings = dict()
  for g_event in artikkel.gold_events:
    if g_event.event_annotation.split(" ")[1] == 'multiword="true"':
      continue
    for embedding in getattr(artikkel, layer_name):
      if embedding.start >= g_event.start and embedding.end <= g_event.end:
        embeddings[g_event.event_ID] = np.asarray(embedding.bert_embedding)
        break
  return embeddings

# Sündmuse fraasi sõnede embeddingute keskmine
# sõnade embedding'ute keskmise arvutamine
def event_embeddingute_keskmised(artikkel, layer_name="bert_embeddings_fixed"):
  embeddings = dict()
  for g_event in artikkel.gold_events:
    for embedding in getattr(artikkel, layer_name):
      if embedding.start >= g_event.start and embedding.end <= g_event.end:
        if g_event.event_ID not in embeddings.keys():
          embeddings[g_event.event_ID] = [embedding.bert_embedding]
        else:
          embeddings[g_event.event_ID].append(embedding.bert_embedding)
  
  embeddings_keskmised = dict()
  for item in embeddings.items():
    keskmine = np.mean(item[1], 0)
    embeddings_keskmised[item[0]] = keskmine
  
  return embeddings_keskmised

# Sündmuse fraasi sõnede embeddingute kaalutatud keskmine
# sõnade embedding'ute kaalutud keskmise arvutamist, kus kõrgem kaal läheb class märgendiga EVENT-ile
def event_embeddingute_kaalutatud_keskmised(artikkel, class_kaal=0.7, muu_kaal=0.3, layer_name="bert_embeddings_fixed"):
  embeddings = dict()
  for g_event in artikkel.gold_events:
    for embedding in getattr(artikkel, layer_name):
      if embedding.start >= g_event.start and embedding.end <= g_event.end:
        if g_event.event_ID not in embeddings.keys():
          embeddings[g_event.event_ID] = [embedding.bert_embedding]
        else:
          embeddings[g_event.event_ID].append(embedding.bert_embedding)
  
  embeddings_keskmised = dict()
  for item in embeddings.items():
    weights = [muu_kaal for i in item[1]]
    # mitmes on CLASS'iga sõne, sellele peab suurema kaalu andma
    mitmes = -1
    for g_event in artikkel.gold_events:
      if g_event.event_ID == item[0]:
        mitmes += 1
        if g_event.event_annotation.split(" ")[1] != 'multiword="true"':
          break
    weights[mitmes] = class_kaal

    keskmine = np.average(item[1], 0, weights)
    embeddings_keskmised[item[0]] = keskmine
  
  return embeddings_keskmised

def leia_sonaliik_ja_tagasta_kaal(artikkel, start, weights):
  pos = -1
  for sone in artikkel.morph_analysis:
    if sone.start == start:
      pos = sone.partofspeech
      if "A" in pos:
        pos = "A"
      else:
        pos = pos[0]
      break

  if pos == "V":
    return weights[0]
  if pos == "A":
    return weights[1]
  if pos in ["S", "A", "C", "U", "O", "N", "H", "P"]:
    return weights[2]
  
  if pos == -1:
    print("Sonaliigis viga!")
  
  return weights[3]
  

# Sündmuse fraasi sõnede embeddingute kaalutatud keskmine sõnaliikide järgi
# sõnade embedding'ute kaalutud keskmise arvutamine, kaalud sõnaliikide prioriteetide järgi
def event_embeddingute_kaalutatud_keskmised_sonaliigid(artikkel, weights=[0.8, 0.7, 0.5, 0.2], layer_name="bert_embeddings_fixed"):
  embeddings = dict()
  starts = dict()
  for g_event in artikkel.gold_events:
    for embedding in getattr(artikkel, layer_name):
      if embedding.start >= g_event.start and embedding.end <= g_event.end:
        if g_event.event_ID not in embeddings.keys():
          embeddings[g_event.event_ID] = [embedding.bert_embedding]
          starts[g_event.event_ID] = [embedding.start]
        else:
          embeddings[g_event.event_ID].append(embedding.bert_embedding)
          starts[g_event.event_ID].append(embedding.start)
  
  embeddings_keskmised = dict()
  for item in embeddings.items():
    kaalud = [0 for i in item[1]]
    mitmes = 0
    for g_event in artikkel.gold_events:
      if g_event.event_ID == item[0] and g_event.start == starts.get(item[0])[mitmes]:
        kaal = leia_sonaliik_ja_tagasta_kaal(artikkel, g_event.start, weights)
        kaalud[mitmes] = kaal
        mitmes += 1
    if artikkel.meta.get("filename") == "aja_pm_2000_10_04.tasak.a029.sol" and item[0] == "e23":
      print("muudan")
      for i in range(len(kaalud)):
        if kaalud[i] == 0:
          kaalud[i] = weights[3]

    if 0 in kaalud:
      print(kaalud)
      print(starts.get(item[0]))
      print(item[1])
      print(artikkel.meta.get("filename"))
    keskmine = np.average(item[1], 0, kaalud)
    embeddings_keskmised[item[0]] = keskmine
  return embeddings_keskmised

# Meetodid artiklist ajaväljendite embeddingute saamiseks

In [12]:
# Sündmuse fraasi sõnede embeddingute keskmine
def timex_embeddingute_keskmised(artikkel, layer_name="bert_embeddings_fixed"):
  embeddings = dict()
  for g_timex in artikkel.gold_timexes_phrases_ambiguous_fixed:
    for embedding in getattr(artikkel, layer_name):
      if embedding.start >= g_timex.start or embedding.end <= g_timex.end:
        # Bert embeddingsis: 19. XI 1997. See koosneb tegelt kahest ajaväljendist 19. XI ja 1997
        for timexID in g_timex.timex_ID:
          if timexID not in embeddings.keys():
            embeddings[timexID] = [embedding.bert_embedding]
          else:
            embeddings[timexID].append(embedding.bert_embedding)
  
  embeddings_keskmised = dict()
  for item in embeddings.items():
    keskmine = np.mean(item[1], 0)
    embeddings_keskmised[item[0]] = keskmine

  return embeddings_keskmised

# Meetodid seostetüüpide embeddingute jaoks

In [13]:
def tlink_event_DCT(eventID):
  return event_embeddings.get(eventID)

def concat_embedding_event_timex(eventID, timexID, event_embeddings, timex_embeddings):
  event = event_embeddings.get(eventID)[0]
  timex = timex_embeddings.get(timexID)[0]
  concat = np.concatenate((event, timex))
  return concat

def concat_embedding_event_event(eventID1, eventID2, event_embeddings):
  event1 = event_embeddings.get(eventID1)[0]
  event2 = event_embeddings.get(eventID2)[0]
  concat = np.concatenate((event1, event2))
  return concat

# Mudel - tlink-event-dct
X - embeddingud  
y - relation type

In [14]:
def loe_relatsioonid():
  data = None
  with open(drive_path + 'relations.pickle', 'rb') as fp:
      #data = json.load(fp)
      data = pickle.load(fp)
  return data
rels = loe_relatsioonid()

In [15]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold

In [16]:
artikkel_event = []

def loo_X_ja_y_DCT(embedding_layer_name, event_embeddings_nr, with_form):
  rels = loe_relatsioonid()
  X = []
  y = []
  
  for artikkel in laetud_artiklid_embeddingutega:
    # Võimalikud kihid: bert_embeddings, bert_embeddings_add, bert_embeddings_all_add, bert_embeddings_lastlayer, bert_embeddings_penultimatelayer

    if event_embeddings_nr == 0:
      event_embeddings = main_event_embedding(artikkel, layer_name=embedding_layer_name)
    elif event_embeddings_nr == 1:
      event_embeddings = event_embeddingute_keskmised(artikkel, layer_name=embedding_layer_name)
    elif event_embeddings_nr == 2:
      event_embeddings = event_embeddingute_kaalutatud_keskmised(artikkel, layer_name=embedding_layer_name)
    elif event_embeddings_nr == 3:
      event_embeddings = event_embeddingute_kaalutatud_keskmised_sonaliigid(artikkel, layer_name=embedding_layer_name)
    
    for item in event_embeddings.items():
      event_relations = rels.get((artikkel.meta.get("filename"), item[0]))
      relation_to_DCT = None
      for relation in event_relations:
        if "t0" in relation:
          relation_to_DCT = relation
      if not relation_to_DCT:
        if False:
          print(artikkel.meta.get("filename"))
          print(event_relations)
          print("DCT relatsioon puudub!")
      else:
        X_data = item[1][0]
        if with_form:
          event_form = None
          start_end = None
          for g_event in artikkel.gold_events:
            if g_event.event_ID == item[0] and g_event.event_annotation.split(" ")[1] != 'multiword="true"':
              start_end = (g_event.start, g_event.end)
              break
          for morph in artikkel.morph_analysis:
            if morph.start == start_end[0] and morph.end == start_end[1]:
              event_form = morph.form[0]
              break
          if event_form:
            X_data = np.append(X_data, event_form)
          else:
            X_data = np.append(X_data, "-")

        X.append(X_data)
        y.append(relation_to_DCT[1])
        artikkel_event.append((artikkel.meta.get("filename"), item[0]))
  
  print(len(X))
  print(len(y))
  return X, y

X, y = loo_X_ja_y_DCT("bert_embeddings_fixed", 0, True)

4010
4010


In [17]:
print(len(X) == len(y))
print(y[0])
print(artikkel_event[0])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

def get_proportions(y_values, osakaal):
  
  y_set = set(y_values)
  vastus = "\n"
  for value in y_set:
    kogus = y_values.count(value)
    if osakaal:
      kogus = round(kogus / len(y_values), 4)
    vastus += value + ": " + str(kogus) + "\n"
  return vastus

print(set(y))
print("Train data class proportions", get_proportions(y_train, osakaal=False))
print("Test data class proportions", get_proportions(y_test, osakaal=False))
print("y data class proportions", get_proportions(y, osakaal=False))

True
OVERLAP-OR-AFTER
('aja_ml_2002_47.tasak.a006.sol', 'e1')
{'AFTER', 'BEFORE-OR-OVERLAP', 'OVERLAP-OR-AFTER', 'INCLUDES', 'VAGUE', 'IS_INCLUDED', 'SIMULTANEOUS', 'BEFORE'}
Train data class proportions 
AFTER: 417
BEFORE-OR-OVERLAP: 350
VAGUE: 468
INCLUDES: 805
OVERLAP-OR-AFTER: 170
IS_INCLUDED: 28
SIMULTANEOUS: 1
BEFORE: 1370

Test data class proportions 
AFTER: 46
BEFORE-OR-OVERLAP: 34
VAGUE: 53
INCLUDES: 98
OVERLAP-OR-AFTER: 20
IS_INCLUDED: 4
BEFORE: 146

y data class proportions 
AFTER: 463
BEFORE-OR-OVERLAP: 384
OVERLAP-OR-AFTER: 190
INCLUDES: 903
VAGUE: 521
IS_INCLUDED: 32
SIMULTANEOUS: 1
BEFORE: 1516



In [18]:
# Kommentaarina on accuracy (f1-score micro) väärtus
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# --- Katsetavad mudelid ---
#clf = make_pipeline(StandardScaler(), LinearSVC(random_state=0, C=0.025))  # 0.59 - main_event_embedding(bert_embeddings)
#clf = make_pipeline(StandardScaler(), SVC(random_state=0, kernel="linear", C=0.025)) # 0.64 - main_event_embedding(bert_embeddings)
#clf = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=len(set(y)) - 2)) # 0.61 - main_event_embedding(bert_embeddings)
#clf = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=5, n_estimators=10, max_features=20, random_state=0)) #0.50 - main_event_embedding(bert_embeddings)
#clf = make_pipeline(StandardScaler(), MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)) # 0.66 - main_event_embedding(bert_embeddings)

# Meetod, et jooksutada mudeli treenimist mitu korda, iga kord muutes mudeli mingit parameetrit.
def jooksuta_ja_muuda_mitu_korda(X, y):
  top1 = [0,0, None]
  top2 = [0,0, None]
  top3 = [0,0, None]

  alpha_value = 0.2
  c_value = 0.05
  for i in range(25):
    clf = make_pipeline(StandardScaler(), SVC(random_state=0, kernel="linear", C=c_value + i * c_value))
    tulemused = ristvalideeri(clf, X, y)

    f1 = f1_score(tulemused[0], tulemused[1], average="micro")
    if f1 > top1[0]:
      top3 = top2
      top2 = top1

      top1[0] = f1
      top1[1] = alpha_value + alpha_value * i
      top1[2] = classification_report(tulemused[0], tulemused[1])
    elif f1 > top2[0]:
      top3 = top2
      top2[0] = f1
      top2[1] = alpha_value + alpha_value * i
      top2[2] = classification_report(tulemused[0], tulemused[1])
    elif f1 > top3[0]:
      top3[0] = f1
      top3[1] = alpha_value + alpha_value * i
      top3[2] = classification_report(tulemused[0], tulemused[1])

    print(f1_score(tulemused[0], tulemused[1], average="micro"), f1_score(tulemused[0], tulemused[1], average="macro"), f1_score(tulemused[0], tulemused[1], average="weighted"), "Alpha value:", alpha_value + alpha_value * i)

  print(top1)
  print(top2)
  print(top3)
  return top1, top2, top3

def ristvalideeri(mudel, X, y):
  all_preds = []
  all_true = []

  all_acc = []
  all_recall = []
  all_prec = []
  all_f1 = []

  all_vabam_acc = []

  highest_accuracy = 0
  highest_accuracy_report = None

  c = list(zip(X, y))
  random.seed(10)
  random.shuffle(c)
  X, y = zip(*c)

  kf = KFold(n_splits=10, shuffle=True, random_state=2021)
  kf.get_n_splits(X)
  i = 0
  for train_index, test_index in kf.split(X):
    print("split", i)
    X_train_split = [X[i] for i in train_index]
    y_train_split = [y[i] for i in train_index]
    X_test_split = [X[i] for i in test_index]
    y_test_split = [y[i] for i in test_index]

    mudel.fit(X_train_split, y_train_split)
    y_pred = mudel.predict(X_test_split)

    all_preds.extend(y_pred)
    all_true.extend(y_test_split)

    all_acc.append(accuracy_score(y_pred, y_test_split))
    all_recall.append(recall_score(y_pred, y_test_split, average="micro"))
    all_prec.append(precision_score(y_pred, y_test_split, average="micro"))
    all_f1.append(f1_score(y_pred, y_test_split, average="micro"))
    all_vabam_acc.append(vabam_accuracy(y_pred, y_test_split))

    if accuracy_score(y_pred, y_test_split) > highest_accuracy:
      highest_accuracy_report = classification_report(y_pred, y_test_split)
      highest_accuracy = accuracy_score(y_pred, y_test_split)

    i += 1
  return (all_preds, all_true, all_acc, all_recall, all_prec, all_f1, highest_accuracy_report, all_vabam_acc)

# Ristvalideerimine ja tulemuste printimine
def prindi_tulemused(mudel, mudeli_nimi, X, y, kihi_nimi):
  tulemused = ristvalideeri(mudel, X, y)
  print("###", kihi_nimi, "-", round(f1_score(tulemused[0], tulemused[1], average="micro"),2), round(f1_score(tulemused[0], tulemused[1], average="macro"),2), round(f1_score(tulemused[0], tulemused[1], average="weighted"),2))
  print("```")
  print(mudeli_nimi)
  print("Parima alamhulga raport:")
  print(tulemused[6])
  print("--- Ristvalideerimise tulemused ---")
  print("Tulemuste keskmised (micro):")
  print("   Accuracy:  ", np.mean(tulemused[2]), "vabam:", np.mean(tulemused[7]))
  print("   Recall:    ", np.mean(tulemused[3]))
  print("   Precision: ", np.mean(tulemused[4]))
  print("   F1-score:  ", np.mean(tulemused[5]))
  print("Üle kõikide ennustuste")
  print("   Accuracy:", accuracy_score(tulemused[0], tulemused[1]), "vabam:", vabam_accuracy(tulemused[0], tulemused[1]))
  print("   Recall:   ", "micro -", recall_score(tulemused[0], tulemused[1], average="micro"))
  print("              macro -", recall_score(tulemused[0], tulemused[1], average="macro"))
  print("           weighted -", recall_score(tulemused[0], tulemused[1], average="weighted"))
  print("   Precision:   ", "micro -", precision_score(tulemused[0], tulemused[1], average="micro"))
  print("                 macro -", precision_score(tulemused[0], tulemused[1], average="macro"))
  print("              weighted -", precision_score(tulemused[0], tulemused[1], average="weighted"))
  print("   F1-score: ", "micro -", f1_score(tulemused[0], tulemused[1], average="micro"))
  print("              macro -", f1_score(tulemused[0], tulemused[1], average="macro"))
  print("           weighted -", f1_score(tulemused[0], tulemused[1], average="weighted"))
  print("```")
  
  return [mudeli_nimi, round(f1_score(tulemused[0], tulemused[1], average="micro"),2), round(f1_score(tulemused[0], tulemused[1], average="macro"),2), round(f1_score(tulemused[0], tulemused[1], average="weighted"),2)]

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Vabama puhul:  
--- õige ennustus = 1  
--- õige 'OVERLAP-OR-AFTER' ja ennustab 'AFTER' = 0.5  
--- õige 'BEFORE-OR-OVERLAP' ja ennustab 'BEFORE' = 0.5  
--- vale ennustus = 0  

In [19]:
def vabam_accuracy(y_pred, y_test):
  y_pred_vabam = []
  for i in range(len(y_pred)):
    if y_pred[i] != y_test[i]:
      if y_pred[i] == 'AFTER' and y_test[i] == 'OVERLAP-OR-AFTER':
        y_pred_vabam.append(0.5)
      elif y_pred[i] == 'BEFORE' and y_test[i] == 'BEFORE-OR-OVERLAP':
        y_pred_vabam.append(0.5)
      else:
        y_pred_vabam.append(0)
    else:
      y_pred_vabam.append(1)

  y_pred_range = []
  for i in range(len(y_pred)):
    if y_pred[i] != y_test[i]:
        y_pred_range.append(0)
    else:
      y_pred_range.append(1)

  return sum(y_pred_vabam) / len(y_pred_vabam)

## Mudelite treenimine erinevate kihtidega ja multiword tüüpidega
Mudelite tulemused salvestatakse hilisemaks uurimiseks

In [ ]:
# Mudelid
clf1 = make_pipeline(LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False))  # 0.59 - main_event_embedding(bert_embeddings)
clf2 = make_pipeline(SVC(random_state=0, kernel="linear", C=0.025)) # 0.64 - main_event_embedding(bert_embeddings)
#clf3 = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=len(set(y)) - 2)) # 0.61 - main_event_embedding(bert_embeddings)
#clf4 = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=5, n_estimators=10, max_features=20, random_state=0)) #0.50 - main_event_embedding(bert_embeddings)
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)) # 0.66 - main_event_embedding(bert_embeddings)

nimed = ['LinearSVC(random_state=0, C=0.025)', 'SVC(random_state=0, kernel="linear", C=0.025)', 'MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)']
mudelid = [clf1,clf2,clf5]

kihid = ["bert_embeddings_fixed", "bert_embeddings_add_fixed", "bert_embeddings_all_add_fixed", "bert_embeddings_lastlayer_fixed", "bert_embeddings_penultimatelayer_fixed"]
# Võimalikud kihid: bert_embeddings, bert_embeddings_add, bert_embeddings_all_add, bert_embeddings_lastlayer, bert_embeddings_penultimatelayer
# Võimalikud multiword viisid: 0 - main_event_embedding
#                              1 - event_embeddingute_keskmised
#                              2 - event_embeddingute_kaalutatud_keskmised
#                              3 - event_embeddingute_kaalutatud_keskmised_sonaliigid

top3 = dict()
multiword_tyyp = {0:"main_event_embedding", 1:"event_embeddingute_keskmised", 2:"event_embeddingute_kaalutatud_keskmised", 3:"event_embeddingute_kaalutatud_keskmised_sonaliigid"}
# Iga multiword tüübi puhul
for i in range(4):
  print("--- multiword tüüp:", multiword_tyyp.get(i))
  # Proovi iga kihiga
  tyyp = dict()
  for kiht in kihid:
    print("------ kiht:", kiht)
    X, y = loo_X_ja_y_DCT(kiht, i)
    # Iga mudeliga
    mudelite_skoorid = []
    for j in range(len(mudelid)):
      clf = mudelid[j]
      m_nimi = nimed[j]
      print(m_nimi)
      mudeli_nimi_ja_f1 = prindi_tulemused(clf, m_nimi, X, y, kiht)
      mudelite_skoorid.append(mudeli_nimi_ja_f1)

    mudelite_skoorid.sort(key = lambda x: x[1], reverse=True)
    
    tyyp[kiht] = mudelite_skoorid[:3]
  
  top3[multiword_tyyp.get(i)] = tyyp

  with open(drive_path + 'top3_DCT.json', 'w') as fp:
    json.dump(top3, fp)

print(top3)

--- multiword tüüp: main_event_embedding
------ kiht: bert_embeddings_fixed
aja_pm_2000_10_04.tasak.a047.sol
[['e19', 'BEFORE', 'e20', ''], ['e18', 'OVERLAP-OR-AFTER', 'e19', '']]
POLE DCT RELATIONIT
aja_sloleht_2007_12_17.tasak.a003.sol
[['e56', 'VAGUE', 'e57', '']]
POLE DCT RELATIONIT
LinearSVC(random_state=0, C=0.025)
split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9
### bert_embeddings_fixed - 0.62 0.41 0.63
```
LinearSVC(random_state=0, C=0.025)
Parima alamhulga raport:
                   precision    recall  f1-score   support

            AFTER       0.64      0.60      0.62        47
           BEFORE       0.81      0.85      0.83       155
BEFORE-OR-OVERLAP       0.67      0.57      0.61        46
         INCLUDES       0.67      0.65      0.66        98
      IS_INCLUDED       0.00      0.00      0.00         1
 OVERLAP-OR-AFTER       0.15      0.11      0.12        19
            VAGUE       0.32      0.40      0.35        35

         accurac

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


   F1-score:  micro - 0.6246882793017456
              macro - 0.40813885458478266
           weighted - 0.6273694258233242
```
SVC(random_state=0, kernel="linear", C=0.025)
split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


split 9
### bert_embeddings_fixed - 0.62 0.41 0.62
```
SVC(random_state=0, kernel="linear", C=0.025)
Parima alamhulga raport:
                   precision    recall  f1-score   support

            AFTER       0.59      0.53      0.56        51
           BEFORE       0.85      0.74      0.79       164
BEFORE-OR-OVERLAP       0.60      0.69      0.64        39
         INCLUDES       0.61      0.68      0.64        84
      IS_INCLUDED       0.00      0.00      0.00         0
 OVERLAP-OR-AFTER       0.29      0.38      0.33        13
            VAGUE       0.46      0.50      0.48        50

         accuracy                           0.65       401
        macro avg       0.49      0.50      0.49       401
     weighted avg       0.68      0.65      0.66       401

--- Ristvalideerimise tulemused ---
Tulemuste keskmised (micro):
   Accuracy:   0.6194513715710723 vabam: 0.6397755610972568
   Recall:     0.6194513715710723
   Precision:  0.6194513715710723
   F1-score:   0.619451371571

...

split 1


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


split 2


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


split 3


In [ ]:
import json
with open(drive_path + 'top3_DCT.json', 'r') as fp:
    top3 = json.load(fp)

In [ ]:
for key in top3.keys():
  print("------------------------ multiword_tüüp:", key)
  for kiht in top3.get(key).keys():
    print("------ kiht:", kiht)
    for mudeli_tulemus in top3.get(key).get(kiht):
      print("### ", mudeli_tulemus[1], mudeli_tulemus[2], mudeli_tulemus[3], "-", mudeli_tulemus[0])
  print()

------------------------ multiword_tüüp: main_event_embedding
------ kiht: bert_embeddings_fixed
###  0.63 0.41 0.64 - MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
###  0.62 0.41 0.63 - LinearSVC(random_state=0, C=0.025)
###  0.62 0.41 0.62 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_add_fixed
###  0.64 0.42 0.65 - MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
###  0.59 0.38 0.59 - LinearSVC(random_state=0, C=0.025)
###  0.59 0.38 0.59 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_all_add_fixed
###  0.63 0.4 0.64 - MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
###  0.59 0.39 0.59 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.56 0.35 0.55 - LinearSVC(random_state=0, C=0.025)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.64 0.42 0.65 - MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
###  0.63 0.42 0.63 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.62 0.41 0.62 - 

## Top3 mudelid erinevate kombinatsioonidega



```
------------------------ multiword_tüüp: main_event_embedding
------ kiht: bert_embeddings_fixed
###  0.63 0.42 0.63 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.63 0.4 0.63 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.62 0.41 0.63 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_add_fixed
###  0.64 0.41 0.64 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.59 0.39 0.59 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.59 0.39 0.6 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_all_add_fixed
###  0.63 0.41 0.64 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.59 0.39 0.59 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.55 0.35 0.54 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.63 0.41 0.64 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.62 0.4 0.62 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.62 0.41 0.62 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.65 0.43 0.66 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.64 0.41 0.65 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.63 0.4 0.64 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)

------------------------ multiword_tüüp: event_embeddingute_keskmised
------ kiht: bert_embeddings_fixed
###  0.63 0.42 0.63 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.62 0.41 0.62 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.62 0.41 0.63 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
------ kiht: bert_embeddings_add_fixed
###  0.63 0.4 0.64 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.59 0.39 0.59 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.58 0.38 0.58 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.64 0.4 0.64 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.6 0.39 0.6 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.56 0.35 0.55 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.63 0.42 0.64 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.62 0.4 0.62 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.62 0.41 0.63 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.65 0.41 0.65 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.64 0.42 0.65 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.63 0.39 0.63 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)

------------------------ multiword_tüüp: event_embeddingute_kaalutatud_keskmised
------ kiht: bert_embeddings_fixed
###  0.63 0.42 0.63 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.63 0.42 0.64 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.62 0.41 0.63 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_add_fixed
###  0.64 0.41 0.65 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.6 0.4 0.6 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.59 0.38 0.58 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.64 0.4 0.64 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.59 0.38 0.59 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.55 0.35 0.55 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.64 0.41 0.64 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.62 0.41 0.62 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.62 0.42 0.63 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.64 0.42 0.65 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.64 0.41 0.65 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.63 0.39 0.64 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)

------------------------ multiword_tüüp: event_embeddingute_kaalutatud_keskmised_sonaliigid
------ kiht: bert_embeddings_fixed
###  0.63 0.42 0.63 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.63 0.42 0.64 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.62 0.41 0.62 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_add_fixed
###  0.64 0.4 0.65 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.59 0.38 0.6 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.58 0.38 0.58 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.64 0.41 0.64 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.59 0.39 0.6 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.56 0.36 0.55 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.64 0.42 0.64 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.62 0.42 0.62 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.62 0.41 0.63 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.64 0.41 0.65 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.64 0.41 0.65 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.63 0.39 0.63 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
```



## Valitud mudel

In [ ]:
X, y = loo_X_ja_y_DCT("bert_embeddings_penultimatelayer_fixed", 0, False)
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=2.8, solver="lbfgs", max_iter=5000, random_state=0))
#clf5 = make_pipeline(StandardScaler(), SVC(random_state=0, kernel="linear", C=0.025, class_weight="balanced"))
tulemused = ristvalideeri(clf5, X, y)
#tulemused = prindi_tulemused(clf5, 'MLPClassifier(alpha=2.8, solver="lbfgs", max_iter=5000, random_state=0)', X, y, "bert_embeddings_penultimatelayer_fixed")

4010
4010
4010
split 0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9


In [ ]:
report = classification_report(tulemused[0], tulemused[1])
print(report)
print("Micro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="micro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="micro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="micro"))
print("Macro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="macro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="macro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="macro"))
print("Weighted")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="weighted"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="weighted"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="weighted"))

print("TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)")
oigeid = 0
for i in range(len(tulemused[0])):
  if tulemused[0][i] == tulemused[1][i]:
    oigeid += 1
print("Skoor:", str(oigeid / len(tulemused[0])))

4010
9
32
0
1
                   precision    recall  f1-score   support

            AFTER       0.51      0.49      0.50       480
           BEFORE       0.85      0.82      0.84      1586
BEFORE-OR-OVERLAP       0.60      0.73      0.66       318
         INCLUDES       0.67      0.63      0.65       963
      IS_INCLUDED       0.16      0.56      0.24         9
 OVERLAP-OR-AFTER       0.16      0.22      0.19       140
     SIMULTANEOUS       0.00      0.00      0.00         0
            VAGUE       0.44      0.45      0.45       514

         accuracy                           0.66      4010
        macro avg       0.43      0.49      0.44      4010
     weighted avg       0.67      0.66      0.66      4010

Micro
- recall: 0.6581047381546135
- precsision: 0.6581047381546135
- f1-score: 0.6581047381546135
Macro


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


- recall: 0.4867538809185814
- precsision: 0.42542095091786153
- f1-score: 0.440641372531819
Weighted
- recall: 0.6581047381546135
- precsision: 0.6716742182407544
- f1-score: 0.6636658436725187
TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)
Skoor: 0.6581047381546135


# Mudel - tlink-event-timex

In [20]:
def loo_X_ja_y_eventtimex(embedding_layer_name, event_embeddings_nr): 
  rels = loe_relatsioonid()
  artikkel_event = []
  X = []
  y = []

  for artikkel in laetud_artiklid_embeddingutega:
    leitud_suhteid = 0
    # Võimalikud kihid: bert_embeddings_fixed, bert_embeddings_add_fixed, bert_embeddings_all_add_fixed, bert_embeddings_lastlayer_fixed, bert_embeddings_penultimatelayer_fixed

    if event_embeddings_nr == 0:
      event_embeddings = main_event_embedding(artikkel, layer_name=embedding_layer_name)
    elif event_embeddings_nr == 1:
      event_embeddings = event_embeddingute_keskmised(artikkel, layer_name=embedding_layer_name)
    elif event_embeddings_nr == 2:
      event_embeddings = event_embeddingute_kaalutatud_keskmised(artikkel, layer_name=embedding_layer_name)
    elif event_embeddings_nr == 3:
      event_embeddings = event_embeddingute_kaalutatud_keskmised_sonaliigid(artikkel, layer_name=embedding_layer_name)

    timex_embeddings = timex_embeddingute_keskmised(artikkel, layer_name=embedding_layer_name)
    for item in event_embeddings.items():
      event_relations = rels.get((artikkel.meta.get("filename"), item[0]))
      leitud = []
      if not event_relations:
        print(artikkel.meta.get("filename"),item[0])
      for relation in event_relations:
        # käi relation läbi ja vaata, kas sisaldab ajaväljendi ID-d (t*)
        if (relation[2] != "t0" and relation[2][0] == "t"):
          timex_ID = None
          if relation[0][0] == "t":
            timex_ID = relation[0]
          else:
            timex_ID = relation[2]
          event_timex_embedding = concat_embedding_event_timex(item[0], timex_ID, event_embeddings, timex_embeddings)
          leitud.append((event_timex_embedding, relation[1]))
      if leitud == []:
        prindi = False
        if prindi:
          print(artikkel.meta.get("filename"))
          print(event_relations)
          print("POLE event_timex RELATIONIT")
      else:
        for leid in leitud:
          X.append(leid[0])
          y.append(leid[1])
          leitud_suhteid += 1
        artikkel_event.append((artikkel.meta.get("filename"), item[0]))

  return X, y

X, y = loo_X_ja_y_eventtimex("bert_embeddings_fixed", 0)

## Kuldmärgendite ja embeddingute arvude kontroll

In [22]:
gt_arv = 0
embt_arv = 0
ge_arv = 0
embe_arv = 0
for art in laetud_artiklid_embeddingutega:
  embs = timex_embeddingute_keskmised(art)
  embt_arv += len(embs.keys())
  gt_arv += len(art.gold_timexes_phrases_ambiguous_fixed)

  eventIDs = set()
  for event in art.gold_events:
    eventIDs.add(event.event_ID)
  ge_arv += len(eventIDs)
  e_main = len(main_event_embedding(art).keys())
  e_kesk = len(event_embeddingute_keskmised(art).keys())
  e_kesk_kaal = len(event_embeddingute_kaalutatud_keskmised(art).keys())
  e_kesk_kaal_son = len(event_embeddingute_kaalutatud_keskmised_sonaliigid(art).keys())

  if not e_main == e_kesk == e_kesk_kaal == e_kesk_kaal_son:
    print(e_main, e_kesk, e_kesk_kaal, e_kesk_kaal_son)
    print(len(art.gold_events))
    break
  else:
    embe_arv += e_main
  
  if ge_arv != embe_arv:
    print(art.meta.get("filename"))
    print(e_main, e_kesk, e_kesk_kaal, e_kesk_kaal_son)
    print(len(art.gold_events))
    break

print("-")
print(gt_arv, embt_arv) # lugemisel tuleb erinevus aja_pm_2000_10_04.tasak.a015.sol-s oleva "1656-1661" tõttu. Kõik on siiski korras.
print(ge_arv, embe_arv)

muudan
-
627 628
4012 4012


## Mudelite treenimine erinevate kihtidega ja multiword tüüpidega

In [ ]:
# Mudelid
clf1 = make_pipeline(StandardScaler(), LinearSVC(random_state=0, C=0.025))  # 0.59 - main_event_embedding(bert_embeddings)
clf2 = make_pipeline(StandardScaler(), SVC(random_state=0, kernel="linear", C=0.025)) # 0.64 - main_event_embedding(bert_embeddings)
clf3 = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=len(set(y)) - 2)) # 0.61 - main_event_embedding(bert_embeddings)
clf4 = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=5, n_estimators=10, max_features=20, random_state=0)) #0.50 - main_event_embedding(bert_embeddings)
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)) # 0.66 - main_event_embedding(bert_embeddings)

nimed = ['LinearSVC(random_state=0, C=0.025)', 'SVC(random_state=0, kernel="linear", C=0.025)', 'KNeighborsClassifier(n_neighbors=len(set(y)) - 2)', 'RandomForestClassifier(max_depth=5, n_estimators=10, max_features=20, random_state=0)', 'MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)']
mudelid = [clf1,clf2,clf3,clf4,clf5]

kihid = ["bert_embeddings_fixed", "bert_embeddings_add_fixed", "bert_embeddings_all_add_fixed", "bert_embeddings_lastlayer_fixed", "bert_embeddings_penultimatelayer_fixed"]
# Võimalikud kihid: bert_embeddings, bert_embeddings_add, bert_embeddings_all_add, bert_embeddings_lastlayer, bert_embeddings_penultimatelayer
# Võimalikud multiword viisid: 0 - main_event_embedding
#                              1 - event_embeddingute_keskmised
#                              2 - event_embeddingute_kaalutatud_keskmised
#                              3 - event_embeddingute_kaalutatud_keskmised_sonaliigid


top3 = dict()
multiword_tyyp = {0:"main_event_embedding", 1:"event_embeddingute_keskmised", 2:"event_embeddingute_kaalutatud_keskmised", 3:"event_embeddingute_kaalutatud_keskmised_sonaliigid"}
# Iga multiword tüübi puhul
for i in range(4):
  print("--- multiword tüüp:", multiword_tyyp.get(i))
  # Proovi iga kihiga
  tyyp = dict()
  for kiht in kihid:
    print("------ kiht:", kiht)
    X, y = loo_X_ja_y_eventtimex(kiht, i)
    # Iga mudeliga
    mudelite_skoorid = []
    for j in range(len(mudelid)):
      clf = mudelid[j]
      m_nimi = nimed[j]
      mudeli_nimi_ja_f1 = prindi_tulemused(clf, m_nimi, X, y, kiht)
      mudelite_skoorid.append(mudeli_nimi_ja_f1)

    mudelite_skoorid.sort(key = lambda x: x[1], reverse=True)
    
    tyyp[kiht] = mudelite_skoorid[:3]
  
  top3[multiword_tyyp.get(i)] = tyyp

print(top3)

--- multiword tüüp: main_event_embedding
------ kiht: bert_embeddings_fixed
559
559


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_star

### bert_embeddings_fixed - 0.53 0.27 0.52
```
LinearSVC(random_state=0, C=0.025)
Parima alamhulga raport:
                   precision    recall  f1-score   support

            AFTER       0.50      0.50      0.50         2
BEFORE-OR-OVERLAP       0.60      0.38      0.46         8
         INCLUDES       0.00      0.00      0.00         2
      IS_INCLUDED       0.70      0.78      0.74        27
 OVERLAP-OR-AFTER       0.33      1.00      0.50         1
     SIMULTANEOUS       0.53      0.50      0.52        16

         accuracy                           0.61        56
        macro avg       0.44      0.53      0.45        56
     weighted avg       0.60      0.61      0.60        56

--- Ristvalideerimise tulemused ---
Tulemuste keskmised (micro):
   Accuracy:   0.5258116883116882 vabam: 0.5258116883116882
   Recall:     0.5258116883116882
   Precision:  0.5258116883116882
   F1-score:   0.5258116883116882
Üle kõikide ennustuste
   Accuracy: 0.5259391771019678 vabam: 0.525939177

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: Und

### bert_embeddings_fixed - 0.55 0.31 0.56
```
SVC(random_state=0, kernel="linear", C=0.025)
Parima alamhulga raport:
                   precision    recall  f1-score   support

            AFTER       1.00      1.00      1.00         1
           BEFORE       0.00      0.00      0.00         0
BEFORE-OR-OVERLAP       0.57      0.57      0.57         7
         INCLUDES       0.00      0.00      0.00         1
      IS_INCLUDED       0.75      0.69      0.72        35
 OVERLAP-OR-AFTER       0.00      0.00      0.00         0
     SIMULTANEOUS       0.43      0.50      0.46        12

         accuracy                           0.62        56
        macro avg       0.39      0.39      0.39        56
     weighted avg       0.65      0.62      0.64        56

--- Ristvalideerimise tulemused ---
Tulemuste keskmised (micro):
   Accuracy:   0.5490584415584415 vabam: 0.5499512987012987
   Recall:     0.5490584415584415
   Precision:  0.5490584415584415
   F1-score:   0.5490584415584415
Üle

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: 

### bert_embeddings_penultimatelayer_fixed - 0.53 0.12 0.64
```
RandomForestClassifier(max_depth=5, n_estimators=10, max_features=20, random_state=0)
Parima alamhulga raport:
                   precision    recall  f1-score   support

            AFTER       0.00      0.00      0.00         0
           BEFORE       0.00      0.00      0.00         0
BEFORE-OR-OVERLAP       0.29      1.00      0.44         2
      IS_INCLUDED       0.97      0.66      0.78        47
 OVERLAP-OR-AFTER       0.00      0.00      0.00         0
     SIMULTANEOUS       0.29      0.57      0.38         7

         accuracy                           0.66        56
        macro avg       0.26      0.37      0.27        56
     weighted avg       0.86      0.66      0.72        56

--- Ristvalideerimise tulemused ---
Tulemuste keskmised (micro):
   Accuracy:   0.5328896103896104 vabam: 0.5328896103896104
   Recall:     0.5328896103896104
   Precision:  0.5328896103896104
   F1-score:   0.5328896103896104
Üle k

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricW

### bert_embeddings_penultimatelayer_fixed - 0.57 0.29 0.59
```
MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
Parima alamhulga raport:
                   precision    recall  f1-score   support

            AFTER       0.00      0.00      0.00         0
           BEFORE       0.00      0.00      0.00         0
BEFORE-OR-OVERLAP       0.00      0.00      0.00         6
         INCLUDES       0.00      0.00      0.00         0
      IS_INCLUDED       0.91      0.72      0.81        40
 OVERLAP-OR-AFTER       0.00      0.00      0.00         0
     SIMULTANEOUS       0.44      0.70      0.54        10

         accuracy                           0.64        56
        macro avg       0.19      0.20      0.19        56
     weighted avg       0.73      0.64      0.67        56

--- Ristvalideerimise tulemused ---
Tulemuste keskmised (micro):
   Accuracy:   0.5723051948051948 vabam: 0.5723051948051948
   Recall:     0.5723051948051948
   Precision:  0.5723051948051948
   F1-scor

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
for key in top3.keys():
  print("------------------------ multiword_tüüp:", key)
  for kiht in top3.get(key).keys():
    print("------ kiht:", kiht)
    for mudeli_tulemus in top3.get(key).get(kiht):
      print("### ", mudeli_tulemus[1], mudeli_tulemus[2], mudeli_tulemus[3], "-", mudeli_tulemus[0])

------------------------ multiword_tüüp: main_event_embedding
------ kiht: bert_embeddings_fixed
###  0.58 0.31 0.6 - MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
###  0.55 0.31 0.56 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.53 0.27 0.52 - LinearSVC(random_state=0, C=0.025)
------ kiht: bert_embeddings_add_fixed
###  0.58 0.31 0.6 - MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
###  0.54 0.31 0.55 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.53 0.13 0.63 - RandomForestClassifier(max_depth=5, n_estimators=10, max_features=20, random_state=0)
------ kiht: bert_embeddings_all_add_fixed
###  0.56 0.3 0.58 - MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
###  0.52 0.11 0.63 - RandomForestClassifier(max_depth=5, n_estimators=10, max_features=20, random_state=0)
###  0.51 0.22 0.52 - LinearSVC(random_state=0, C=0.025)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.57 0.3 0.58 - MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
##

## Top3 mudelid erinevate kombinatsioonidega



```
------------------------ multiword_tüüp: main_event_embedding
------ kiht: bert_embeddings_fixed
###  0.58 0.29 0.6 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.55 0.3 0.56 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.52 0.27 0.52 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_add_fixed
###  0.58 0.31 0.6 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.54 0.31 0.55 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.52 0.24 0.53 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.57 0.3 0.58 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.51 0.23 0.52 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.51 0.27 0.51 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.57 0.3 0.58 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.53 0.3 0.54 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.52 0.27 0.52 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.57 0.3 0.58 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.52 0.25 0.53 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.51 0.23 0.52 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------------------------ multiword_tüüp: event_embeddingute_keskmised
------ kiht: bert_embeddings_fixed
###  0.58 0.3 0.6 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.53 0.27 0.54 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.52 0.29 0.52 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_add_fixed
###  0.56 0.3 0.57 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.53 0.24 0.54 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.53 0.27 0.54 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_all_add_fixed
###  0.56 0.29 0.57 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.52 0.25 0.53 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.51 0.26 0.51 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.55 0.29 0.57 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.53 0.28 0.53 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.53 0.29 0.53 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.55 0.29 0.57 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.51 0.25 0.52 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.5 0.23 0.51 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------------------------ multiword_tüüp: event_embeddingute_kaalutatud_keskmised
------ kiht: bert_embeddings_fixed
###  0.57 0.29 0.59 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.54 0.27 0.55 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.52 0.28 0.52 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_add_fixed
###  0.58 0.32 0.6 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.54 0.29 0.55 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.53 0.24 0.53 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.56 0.29 0.58 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.52 0.23 0.52 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.49 0.24 0.5 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.57 0.28 0.58 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.53 0.28 0.53 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.53 0.3 0.54 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.56 0.29 0.58 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.52 0.26 0.53 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.5 0.22 0.5 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------------------------ multiword_tüüp: event_embeddingute_kaalutatud_keskmised_sonaliigid
------ kiht: bert_embeddings_fixed
###  0.58 0.3 0.6 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.53 0.29 0.53 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.53 0.27 0.55 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_add_fixed
###  0.57 0.3 0.58 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.54 0.28 0.55 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.52 0.24 0.53 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.55 0.29 0.57 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.52 0.25 0.53 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.5 0.27 0.5 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.56 0.29 0.57 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.53 0.28 0.53 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.52 0.29 0.53 - SVC(random_state=0, kernel="linear", C=0.025)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.57 0.29 0.59 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.52 0.26 0.52 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.51 0.23 0.51 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
```



## Valitud mudel

In [ ]:
X, y = loo_X_ja_y_eventtimex("bert_embeddings_add_fixed", 0)
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=3, solver="sgd", max_iter=10000, random_state=0))
#clf5 = make_pipeline(StandardScaler(), SVC(random_state=0, kernel="linear", C=0.025, class_weight="balanced"))
tulemused = ristvalideeri(clf5, X, y)
#tulemused = prindi_tulemused(clf5, 'MLPClassifier(alpha=3, solver="sgd", max_iter=10000, random_state=0)', X, y, "bert_embeddings_penultimatelayer_fixed")

559
559
559
split 0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


split 1


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9


In [ ]:
report = classification_report(tulemused[0], tulemused[1])
print(report)
print("Micro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="micro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="micro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="micro"))
print("Macro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="macro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="macro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="macro"))
print("Weighted")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="weighted"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="weighted"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="weighted"))

print("TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)")
oigeid = 0
for i in range(len(tulemused[0])):
  if tulemused[0][i] == tulemused[1][i]:
    oigeid += 1
print("Skoor:", str(oigeid / len(tulemused[0])))

559
341
293
138
148
                   precision    recall  f1-score   support

            AFTER       0.21      0.30      0.25        10
           BEFORE       0.00      0.00      0.00         0
BEFORE-OR-OVERLAP       0.33      0.36      0.34        59
         INCLUDES       0.25      0.67      0.36         3
      IS_INCLUDED       0.80      0.69      0.74       341
 OVERLAP-OR-AFTER       0.15      0.50      0.24         8
     SIMULTANEOUS       0.48      0.51      0.50       138
            VAGUE       0.00      0.00      0.00         0

         accuracy                           0.60       559
        macro avg       0.28      0.38      0.30       559
     weighted avg       0.65      0.60      0.62       559

Micro
- recall: 0.5992844364937389
- precsision: 0.5992844364937389
- f1-score: 0.5992844364937389
Macro
- recall: 0.37791357905966744
- precsision: 0.27872871793520254
- f1-score: 0.3034833274840051
Weighted
- recall: 0.5992844364937389
- precsision: 0.648170781275056

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Mudel - tlink-main-events

loe_main_relatsioonid() meetodiga saadud sõnastikust saab (failinimi, ID) kaudu kätte kõik relatsioonid, kus selle ID'ga sündmus/ajaväljend on.  
See tähendab, et kui "ajaleht2" sündmused "e2" ja "e3" on suhtes, siis ("ajaleht2", "e2") ja ("ajaleht2", "e3") sisaldavad samasugust relatsiooni.  
Korduvused tuleb õppimise jaoks eemaldada.

In [31]:
def loe_main_relatsioonid():
  data = None
  with open(drive_path + 'relations_only_main.pickle', 'rb') as fp:
      #data = json.load(fp)
      data = pickle.load(fp)
  return data
main_rels = loe_main_relatsioonid()

koik = []
for key in main_rels:
  koik.extend(main_rels.get(key))
print("Iga evendi kõik relatsioonid:", len(koik))

def main_relatsioonid_korduvusteta():
  main_rels = loe_main_relatsioonid()
  # korduvused välja
  uus_dict = {}
  #kasutatud_rels = []
  for key in main_rels:
    relations = main_rels.get(key)
    sobivad = []
    for rel in relations:
      if rel[0] == key[1]:
        sobivad.append(rel)
    uus_dict[key] = sobivad
  return uus_dict

uus_dict = main_relatsioonid_korduvusteta()

koik_uus = []
for key in uus_dict:
  koik_uus.extend(uus_dict.get(key))
print("Relatsioonide kordused väljas:", len(koik_uus))


names = []
for art in laetud_artiklid_embeddingutega:
  names.append(art.meta.get("filename"))

kor = []
for key in uus_dict:
  if key[0] in names:
    kor.extend(uus_dict.get(key))
print("Korras artiklite omi:", len(kor))


Iga evendi kõik relatsioonid: 5104
Relatsioonide kordused väljas: 2552
Korras artiklite omi: 2552


In [32]:
def loo_X_ja_y_main_events (embedding_layer_name, event_embeddings_nr): 
  main_relatsioonid = main_relatsioonid_korduvusteta()
  artikkel_event = []
  X = []
  y = []

  for artikkel in laetud_artiklid_embeddingutega:
    leitud_suhteid = 0
    # Võimalikud kihid: bert_embeddings, bert_embeddings_add, bert_embeddings_all_add, bert_embeddings_lastlayer, bert_embeddings_penultimatelayer

    if event_embeddings_nr == 0:
      event_embeddings = main_event_embedding(artikkel, layer_name=embedding_layer_name)
    elif event_embeddings_nr == 1:
      event_embeddings = event_embeddingute_keskmised(artikkel, layer_name=embedding_layer_name)
    elif event_embeddings_nr == 2:
      event_embeddings = event_embeddingute_kaalutatud_keskmised(artikkel, layer_name=embedding_layer_name)
    elif event_embeddings_nr == 3:
      event_embeddings = event_embeddingute_kaalutatud_keskmised_sonaliigid(artikkel, layer_name=embedding_layer_name)

    for item in event_embeddings.items():
      event_relations = main_relatsioonid.get((artikkel.meta.get("filename"), item[0]))
      leitud = []
      if not event_relations:
        continue
      for relation in event_relations:
        event_event_embedding = concat_embedding_event_event(item[0], relation[2], event_embeddings)
        leitud.append((event_event_embedding, relation[1]))
      if leitud == []:
        prindi = False
        if prindi:
          print(artikkel.meta.get("filename"))
          print(event_relations)
          print("POLE event_timex RELATIONIT")
      else:
        for leid in leitud:
          X.append(leid[0])
          y.append(leid[1])
          leitud_suhteid += 1
        artikkel_event.append((artikkel.meta.get("filename"), item[0]))
  return X, y

X, y = loo_X_ja_y_main_events("bert_embeddings_fixed", 0)

## Mudelite treenimine erinevate kihtidega ja multiword tüüpidega

In [ ]:
# Mudelid
clf1 = make_pipeline(StandardScaler(), LinearSVC(random_state=0, C=0.025, max_iter=2000, dual=False))  # 0.59 - main_event_embedding(bert_embeddings)
clf2 = make_pipeline(StandardScaler(), SVC(random_state=0, kernel="linear", C=0.025)) # 0.64 - main_event_embedding(bert_embeddings)
#clf3 = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=len(set(y)) - 2)) # 0.61 - main_event_embedding(bert_embeddings)
#clf4 = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=5, n_estimators=10, max_features=20, random_state=0)) #0.50 - main_event_embedding(bert_embeddings)
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=0.2, max_iter=2000, random_state=0)) # 0.66 - main_event_embedding(bert_embeddings)

nimed = ['LinearSVC(random_state=0, C=0.025)', 'SVC(random_state=0, kernel="linear", C=0.025)', 'MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)']
mudelid = [clf1,clf2,clf5]

kihid = ["bert_embeddings_fixed", "bert_embeddings_add_fixed", "bert_embeddings_all_add_fixed", "bert_embeddings_lastlayer_fixed", "bert_embeddings_penultimatelayer_fixed"]
# Võimalikud kihid: bert_embeddings, bert_embeddings_add, bert_embeddings_all_add, bert_embeddings_lastlayer, bert_embeddings_penultimatelayer
# Võimalikud multiword viisid: 0 - main_event_embedding
#                              1 - event_embeddingute_keskmised
#                              2 - event_embeddingute_kaalutatud_keskmised
#                              3 - event_embeddingute_kaalutatud_keskmised_sonaliigid

top3 = dict()
multiword_tyyp = {0:"main_event_embedding", 1:"event_embeddingute_keskmised", 2:"event_embeddingute_kaalutatud_keskmised", 3:"event_embeddingute_kaalutatud_keskmised_sonaliigid"}
# Iga multiword tüübi puhul
for i in range(4):
  print("--- multiword tüüp:", multiword_tyyp.get(i))
  # Proovi iga kihiga
  tyyp = dict()
  for kiht in kihid:
    print("------ kiht:", kiht)
    X, y = loo_X_ja_y_main_events(kiht, i)
    # Iga mudeliga
    mudelite_skoorid = []
    for j in range(len(mudelid)):
      clf = mudelid[j]
      m_nimi = nimed[j]
      mudeli_nimi_ja_f1 = prindi_tulemused(clf, m_nimi, X, y, kiht)
      mudelite_skoorid.append(mudeli_nimi_ja_f1)

    mudelite_skoorid.sort(key = lambda x: x[1], reverse=True)
    
    tyyp[kiht] = mudelite_skoorid[:3]
  
  top3[multiword_tyyp.get(i)] = tyyp

  with open(drive_path + 'top3_main_events.json', 'w') as fp:
    json.dump(top3, fp)

print(top3)

--- multiword tüüp: main_event_embedding
------ kiht: bert_embeddings_fixed
2552
2552


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### bert_embeddings_fixed - 0.47 0.38 0.47
```
LinearSVC(random_state=0, C=0.025)
Parima alamhulga raport:
                   precision    recall  f1-score   support

            AFTER       0.53      0.54      0.54        35
           BEFORE       0.71      0.64      0.68        70
BEFORE-OR-OVERLAP       0.37      0.39      0.38        18
         IDENTITY       0.00      0.00      0.00         0
         INCLUDES       0.36      0.34      0.35        29
      IS_INCLUDED       0.53      0.50      0.51        36
 OVERLAP-OR-AFTER       0.18      0.33      0.24         6
     SIMULTANEOUS       0.42      0.23      0.29        22
            VAGUE       0.47      0.62      0.53        39

         accuracy                           0.51       255
        macro avg       0.40      0.40      0.39       255
     weighted avg       0.52      0.51      0.51       255

--- Ristvalideerimise tulemused ---
Tulemuste keskmised (micro):
   Accuracy:   0.4670986519607843 vabam: 0.483356311274509

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### bert_embeddings_fixed - 0.49 0.41 0.5
```
SVC(random_state=0, kernel="linear", C=0.025)
Parima alamhulga raport:
                   precision    recall  f1-score   support

            AFTER       0.68      0.61      0.64        46
           BEFORE       0.69      0.57      0.62        61
BEFORE-OR-OVERLAP       0.45      0.40      0.43        25
         IDENTITY       0.00      0.00      0.00         0
         INCLUDES       0.56      0.57      0.56        35
      IS_INCLUDED       0.50      0.53      0.52        30
 OVERLAP-OR-AFTER       0.38      0.38      0.38        13
     SIMULTANEOUS       0.18      0.31      0.23        13
            VAGUE       0.56      0.62      0.59        32

         accuracy                           0.54       255
        macro avg       0.44      0.44      0.44       255
     weighted avg       0.57      0.54      0.55       255

--- Ristvalideerimise tulemused ---
Tulemuste keskmised (micro):
   Accuracy:   0.49178615196078435 vabam: 0.5123

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### bert_embeddings_penultimatelayer_fixed - 0.48 0.38 0.48
```
SVC(random_state=0, kernel="linear", C=0.025)
Parima alamhulga raport:
                   precision    recall  f1-score   support

            AFTER       0.76      0.51      0.61        49
           BEFORE       0.62      0.63      0.63        52
BEFORE-OR-OVERLAP       0.21      0.29      0.24        14
         IDENTITY       0.00      0.00      0.00         2
         INCLUDES       0.50      0.56      0.53        32
      IS_INCLUDED       0.56      0.71      0.63        31
 OVERLAP-OR-AFTER       0.20      0.20      0.20        15
     SIMULTANEOUS       0.35      0.35      0.35        23
            VAGUE       0.57      0.54      0.56        37

         accuracy                           0.52       255
        macro avg       0.42      0.42      0.42       255
     weighted avg       0.54      0.52      0.53       255

--- Ristvalideerimise tulemused ---
Tulemuste keskmised (micro):
   Accuracy:   0.4773023897058

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### bert_embeddings_penultimatelayer_fixed - 0.47 0.4 0.48
```
MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
Parima alamhulga raport:
                   precision    recall  f1-score   support

            AFTER       0.61      0.69      0.65        32
           BEFORE       0.65      0.60      0.63        68
BEFORE-OR-OVERLAP       0.42      0.35      0.38        23
         IDENTITY       0.00      0.00      0.00         1
         INCLUDES       0.54      0.41      0.46        37
      IS_INCLUDED       0.65      0.65      0.65        34
 OVERLAP-OR-AFTER       0.27      0.30      0.29        10
     SIMULTANEOUS       0.42      0.28      0.33        18
            VAGUE       0.45      0.72      0.55        32

         accuracy                           0.55       255
        macro avg       0.45      0.44      0.44       255
     weighted avg       0.55      0.55      0.54       255

--- Ristvalideerimise tulemused ---
Tulemuste keskmised (micro):
   Accuracy:   0.4745

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
with open(drive_path + 'top3_main_events.json', 'r') as fp:
    top3 = json.load(fp)

In [ ]:
for key in top3.keys():
  print("------------------------ multiword_tüüp:", key)
  for kiht in top3.get(key).keys():
    print("------ kiht:", kiht)
    for mudeli_tulemus in top3.get(key).get(kiht):
      print("### ", mudeli_tulemus[1], mudeli_tulemus[2], mudeli_tulemus[3], "-", mudeli_tulemus[0])
  print()

------------------------ multiword_tüüp: main_event_embedding
------ kiht: bert_embeddings_fixed
###  0.49 0.41 0.5 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.47 0.38 0.47 - LinearSVC(random_state=0, C=0.025)
###  0.46 0.37 0.46 - MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
------ kiht: bert_embeddings_add_fixed
###  0.48 0.39 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.48 0.41 0.49 - MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
###  0.44 0.35 0.45 - LinearSVC(random_state=0, C=0.025)
------ kiht: bert_embeddings_all_add_fixed
###  0.47 0.4 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.47 0.4 0.48 - MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
###  0.45 0.38 0.45 - LinearSVC(random_state=0, C=0.025)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.48 0.39 0.49 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.46 0.37 0.47 - MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)
###  0.44 0.36 0.44 - Li

## Top3 mudelid erinvate kombinatsioonidega



```
------------------------ multiword_tüüp: main_event_embedding
------ kiht: bert_embeddings_fixed
###  0.49 0.41 0.5 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.47 0.38 0.47 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.47 0.38 0.47 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
------ kiht: bert_embeddings_add_fixed
###  0.5 0.42 0.51 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.47 0.39 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.45 0.36 0.45 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.47 0.39 0.47 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.47 0.37 0.47 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.45 0.37 0.45 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.49 0.39 0.49 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.48 0.39 0.49 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.44 0.35 0.44 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.49 0.41 0.5 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.48 0.4 0.49 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.44 0.35 0.44 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)

------------------------ multiword_tüüp: event_embeddingute_keskmised
------ kiht: bert_embeddings_fixed
###  0.49 0.4 0.49 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.46 0.37 0.46 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.46 0.38 0.47 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
------ kiht: bert_embeddings_add_fixed
###  0.48 0.4 0.49 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.47 0.39 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.44 0.35 0.45 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.47 0.39 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.47 0.4 0.47 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.43 0.36 0.43 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.48 0.4 0.48 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.47 0.38 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.44 0.35 0.44 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.49 0.4 0.49 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.47 0.39 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.44 0.35 0.44 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)

------------------------ multiword_tüüp: event_embeddingute_kaalutatud_keskmised
------ kiht: bert_embeddings_fixed
###  0.49 0.41 0.5 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.48 0.4 0.48 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.47 0.38 0.47 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_add_fixed
###  0.48 0.4 0.48 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.47 0.39 0.47 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.44 0.35 0.45 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.49 0.41 0.5 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.47 0.39 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.44 0.37 0.44 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.48 0.38 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.48 0.39 0.49 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.43 0.35 0.43 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.47 0.39 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.47 0.39 0.48 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.43 0.34 0.43 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)

------------------------ multiword_tüüp: event_embeddingute_kaalutatud_keskmised_sonaliigid
------ kiht: bert_embeddings_fixed
###  0.49 0.4 0.5 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.47 0.37 0.47 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
###  0.47 0.38 0.47 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
------ kiht: bert_embeddings_add_fixed
###  0.49 0.4 0.49 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.47 0.39 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.45 0.35 0.45 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.47 0.39 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.47 0.39 0.48 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.44 0.36 0.44 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.48 0.38 0.48 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.47 0.38 0.48 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.44 0.35 0.44 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.47 0.38 0.47 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.47 0.37 0.47 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.43 0.35 0.43 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
```



## Valitud mudel

In [ ]:
X, y = loo_X_ja_y_main_events("bert_embeddings_add_fixed", 0)
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=2.8, solver="lbfgs", max_iter=10000, random_state=0))
#clf5 = make_pipeline(StandardScaler(), SVC(random_state=0, kernel="linear", C=0.025, class_weight="balanced"))
tulemused = ristvalideeri(clf5, X, y)
#tulemused = prindi_tulemused(clf5, 'MLPClassifier(alpha=2.8, solver="lbfgs", max_iter=10000, random_state=0)', X, y, "bert_embeddings_penultimatelayer_fixed")

2552
2552
2552
split 0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


split 1
split 2


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


split 3
split 4
split 5
split 6
split 7
split 8
split 9


In [ ]:
report = classification_report(tulemused[0], tulemused[1])
print(report)
print("Micro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="micro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="micro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="micro"))
print("Macro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="macro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="macro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="macro"))
print("Weighted")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="weighted"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="weighted"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="weighted"))

print("TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)")
oigeid = 0
for i in range(len(tulemused[0])):
  if tulemused[0][i] == tulemused[1][i]:
    oigeid += 1
print("Skoor:", str(oigeid / len(tulemused[0])))

2552
336
321
211
228
                   precision    recall  f1-score   support

            AFTER       0.60      0.55      0.57       405
           BEFORE       0.67      0.59      0.63       637
BEFORE-OR-OVERLAP       0.28      0.38      0.32       145
         IDENTITY       0.09      1.00      0.17         1
         INCLUDES       0.48      0.50      0.49       315
      IS_INCLUDED       0.53      0.51      0.52       336
 OVERLAP-OR-AFTER       0.20      0.26      0.23       120
     SIMULTANEOUS       0.35      0.37      0.36       211
            VAGUE       0.56      0.57      0.57       382

         accuracy                           0.51      2552
        macro avg       0.42      0.52      0.43      2552
     weighted avg       0.53      0.51      0.52      2552

Micro
- recall: 0.5117554858934169
- precsision: 0.5117554858934169
- f1-score: 0.5117554858934169
Macro
- recall: 0.524471734955591
- precsision: 0.41771222580227113
- f1-score: 0.42722828056311407
Weighted
-

# Mudel - tlink-subordinate-events

In [27]:
def loe_sub_relatsioonid():
  data = None
  with open(drive_path + 'relations_only_sub.pickle', 'rb') as fp:
      #data = json.load(fp)
      data = pickle.load(fp)
  return data
sub_rels = loe_sub_relatsioonid()

koik = []
for key in sub_rels:
  koik.extend(sub_rels.get(key))
print("Iga evendi kõik relatsioonid:", len(koik))

def sub_relatsioonid_korduvusteta():
  sub_rels = loe_sub_relatsioonid()
  # korduvused välja
  uus_dict = {}
  for key in sub_rels:
    relations = sub_rels.get(key)
    sobivad = []
    for rel in relations:
      if rel[0] == key[1]:
        sobivad.append(rel)
    uus_dict[key] = sobivad
  return uus_dict

uus_dict = sub_relatsioonid_korduvusteta()

koik_uus = []
for key in uus_dict:
  koik_uus.extend(uus_dict.get(key))
print("Relatsioonide kordused väljas:", len(koik_uus))


names = []
for art in laetud_artiklid_embeddingutega:
  names.append(art.meta.get("filename"))

kor = []
for key in uus_dict:
  if key[0] in names:
    kor.extend(uus_dict.get(key))
print("Korras artiklite omi:", len(kor))

Iga evendi kõik relatsioonid: 6254
Relatsioonide kordused väljas: 3127
Korras artiklite omi: 3127


In [30]:
def loo_X_ja_y_sub_events(embedding_layer_name, event_embeddings_nr): 
  main_relatsioonid = sub_relatsioonid_korduvusteta()
  artikkel_event = []
  X = []
  y = []

  for artikkel in laetud_artiklid_embeddingutega:
    leitud_suhteid = 0
    # Võimalikud kihid: bert_embeddings, bert_embeddings_add, bert_embeddings_all_add, bert_embeddings_lastlayer, bert_embeddings_penultimatelayer

    if event_embeddings_nr == 0:
      event_embeddings = main_event_embedding(artikkel, layer_name=embedding_layer_name)
    elif event_embeddings_nr == 1:
      event_embeddings = event_embeddingute_keskmised(artikkel, layer_name=embedding_layer_name)
    elif event_embeddings_nr == 2:
      event_embeddings = event_embeddingute_kaalutatud_keskmised(artikkel, layer_name=embedding_layer_name)
    elif event_embeddings_nr == 3:
      event_embeddings = event_embeddingute_kaalutatud_keskmised_sonaliigid(artikkel, layer_name=embedding_layer_name)

    for item in event_embeddings.items():
      event_relations = main_relatsioonid.get((artikkel.meta.get("filename"), item[0]))
      leitud = []
      if not event_relations:
        continue
      for relation in event_relations:
        event_event_embedding = concat_embedding_event_event(item[0], relation[2], event_embeddings)
        leitud.append((event_event_embedding, relation[1]))
      if leitud == []:
        prindi = False
        if prindi:
          print(artikkel.meta.get("filename"))
          print(event_relations)
          print("POLE event_event RELATIONIT")
      else:
        for leid in leitud:
          X.append(leid[0])
          y.append(leid[1])
          leitud_suhteid += 1
        artikkel_event.append((artikkel.meta.get("filename"), item[0]))

  return X, y

X, y = loo_X_ja_y_sub_events("bert_embeddings_fixed", 0)

## Mudelite treenimine erinevate kihtidega ja multiword tüüpidega

In [ ]:
# Mudelid
clf1 = make_pipeline(LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=20000, dual=False))  # 0.59 - main_event_embedding(bert_embeddings)
clf2 = make_pipeline(SVC(random_state=0, kernel="linear", C=0.025)) # 0.64 - main_event_embedding(bert_embeddings)
#clf3 = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=len(set(y)) - 2)) # 0.61 - main_event_embedding(bert_embeddings)
#clf4 = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=5, n_estimators=10, max_features=20, random_state=0)) #0.50 - main_event_embedding(bert_embeddings)
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=0.2, max_iter=2000, random_state=0)) # 0.66 - main_event_embedding(bert_embeddings)

nimed = ['LinearSVC(random_state=0, C=0.025)', 'SVC(random_state=0, kernel="linear", C=0.025)', 'MLPClassifier(alpha=0.2, max_iter=1000, random_state=0)']
mudelid = [clf1,clf2,clf5]

kihid = ["bert_embeddings_fixed", "bert_embeddings_add_fixed", "bert_embeddings_all_add_fixed", "bert_embeddings_lastlayer_fixed", "bert_embeddings_penultimatelayer_fixed"]
# Võimalikud kihid: bert_embeddings, bert_embeddings_add, bert_embeddings_all_add, bert_embeddings_lastlayer, bert_embeddings_penultimatelayer
# Võimalikud multiword viisid: 0 - main_event_embedding
#                              1 - event_embeddingute_keskmised
#                              2 - event_embeddingute_kaalutatud_keskmised
#                              3 - event_embeddingute_kaalutatud_keskmised_sonaliigid

top3 = dict()
multiword_tyyp = {0:"main_event_embedding", 1:"event_embeddingute_keskmised", 2:"event_embeddingute_kaalutatud_keskmised", 3:"event_embeddingute_kaalutatud_keskmised_sonaliigid"}
# Iga multiword tüübi puhul
for i in range(4):
  print("--- multiword tüüp:", multiword_tyyp.get(i))
  # Proovi iga kihiga
  tyyp = dict()
  for kiht in kihid:
    print("------ kiht:", kiht)
    X, y = loo_X_ja_y_sub_events(kiht, i)
    # Iga mudeliga
    mudelite_skoorid = []
    for j in range(len(mudelid)):
      clf = mudelid[j]
      m_nimi = nimed[j]
      print(m_nimi)
      mudeli_nimi_ja_f1 = prindi_tulemused(clf, m_nimi, X, y, kiht)
      mudelite_skoorid.append(mudeli_nimi_ja_f1)

    mudelite_skoorid.sort(key = lambda x: x[1], reverse=True)
    
    tyyp[kiht] = mudelite_skoorid[:3]
  
  top3[multiword_tyyp.get(i)] = tyyp

  with open(drive_path + 'top3_main_events.json', 'w') as fp:
    json.dump(top3, fp)

print(top3)

--- multiword tüüp: main_event_embedding
------ kiht: bert_embeddings_fixed
3127
3127
LinearSVC(random_state=0, C=0.025)
split 0


KeyboardInterrupt: ignored

In [ ]:
for key in top3.keys():
  print("------------------------ multiword_tüüp:", key)
  for kiht in top3.get(key).keys():
    print("------ kiht:", kiht)
    for mudeli_tulemus in top3.get(key).get(kiht):
      print("### ", mudeli_tulemus[1], mudeli_tulemus[2], mudeli_tulemus[3], "-", mudeli_tulemus[0])
  print()

## Top3 mudelid erinevate kombinatsioonidega



```
------------------------ multiword_tüüp: main_event_embedding
------ kiht: bert_embeddings_fixed
###  0.44 0.36 0.45 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.42 0.35 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.39 0.32 0.39 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_add_fixed
###  0.44 0.36 0.44 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.42 0.35 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.38 0.31 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.44 0.36 0.44 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.43 0.35 0.44 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.37 0.31 0.37 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.43 0.35 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.4 0.33 0.41 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.37 0.31 0.37 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.43 0.35 0.44 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.41 0.34 0.42 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.38 0.31 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)

------------------------ multiword_tüüp: event_embeddingute_keskmised
------ kiht: bert_embeddings_fixed
###  0.44 0.37 0.45 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.42 0.34 0.42 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.39 0.33 0.4 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_add_fixed
###  0.44 0.36 0.45 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.43 0.35 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.38 0.31 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.43 0.35 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.43 0.35 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.38 0.31 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.43 0.35 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.4 0.33 0.41 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.36 0.3 0.36 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.45 0.37 0.46 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.43 0.35 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.38 0.32 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)

------------------------ multiword_tüüp: event_embeddingute_kaalutatud_keskmised
------ kiht: bert_embeddings_fixed
###  0.44 0.36 0.45 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.42 0.35 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.4 0.33 0.4 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_add_fixed
###  0.44 0.37 0.45 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.42 0.34 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.37 0.31 0.37 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.44 0.36 0.44 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.43 0.36 0.44 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.37 0.31 0.37 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.43 0.35 0.44 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.4 0.33 0.41 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.36 0.3 0.36 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.45 0.37 0.45 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.42 0.34 0.42 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.38 0.32 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)

------------------------ multiword_tüüp: event_embeddingute_kaalutatud_keskmised_sonaliigid
------ kiht: bert_embeddings_fixed
###  0.44 0.36 0.45 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.41 0.34 0.41 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.39 0.32 0.4 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_add_fixed
###  0.44 0.36 0.44 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.42 0.35 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.37 0.31 0.37 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.43 0.35 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.43 0.36 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.38 0.31 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.43 0.35 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.4 0.33 0.4 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.36 0.3 0.36 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.45 0.37 0.45 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.42 0.34 0.42 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.38 0.32 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
```



## Valitud mudel

In [ ]:
X, y = loo_X_ja_y_sub_events("bert_embeddings_penultimatelayer_fixed", 1)
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=1, solver="lbfgs", max_iter=10000, random_state=0))
#clf5 = make_pipeline(StandardScaler(), SVC(random_state=0, kernel="linear", C=1))
tulemused = ristvalideeri(clf5, X, y)
#tulemused = prindi_tulemused(clf5, 'MLPClassifier(alpha=1, solver="lbfgs", max_iter=10000, random_state=0)', X, y, "bert_embeddings_penultimatelayer_fixed")

3127
3127
3127
split 0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


split 1
split 2
split 3
split 4
split 5
split 6
split 7


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


split 8
split 9


In [ ]:
report = classification_report(tulemused[0], tulemused[1])
print(report)
print("Micro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="micro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="micro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="micro"))
print("Macro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="macro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="macro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="macro"))
print("Weighted")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="weighted"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="weighted"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="weighted"))

print("TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)")
oigeid = 0
for i in range(len(tulemused[0])):
  if tulemused[0][i] == tulemused[1][i]:
    oigeid += 1
print("Skoor:", str(oigeid / len(tulemused[0])))

3127
278
304
283
318
                   precision    recall  f1-score   support

            AFTER       0.56      0.54      0.55       399
           BEFORE       0.62      0.55      0.58       728
BEFORE-OR-OVERLAP       0.38      0.41      0.39       395
         IDENTITY       0.00      0.00      0.00         0
         INCLUDES       0.43      0.44      0.43       355
      IS_INCLUDED       0.41      0.45      0.43       278
 OVERLAP-OR-AFTER       0.23      0.28      0.25       120
     SIMULTANEOUS       0.36      0.41      0.39       283
            VAGUE       0.43      0.41      0.42       569

         accuracy                           0.46      3127
        macro avg       0.38      0.39      0.38      3127
     weighted avg       0.47      0.46      0.46      3127

Micro
- recall: 0.4595458906299968
- precsision: 0.4595458906299968
- f1-score: 0.45954589062999684
Macro
- recall: 0.3859220308387335
- precsision: 0.3799646927619876
- f1-score: 0.38221937080010343
Weighted


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


- recall: 0.4595458906299968
- precsision: 0.4679755893418784
- f1-score: 0.4629301383525842
TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)
Skoor: 0.4595458906299968


# Hilisemad katsetused

## Mudel - kõik koos

In [ ]:
# Eraldan igast suhterühmast testimiseks andmeid, et katsetada eraldi nende tulemusi.
X_train_data = []
y_train_data = []

layer = "bert_embeddings_penultimatelayer_fixed"
viis = 0

X, y = loo_X_ja_y_DCT(layer, viis, False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train_data.extend(X_train)
y_train_data.extend(y_train)
event_dct_data = (X_test, y_test)
X, y = loo_X_ja_y_eventtimex(layer, viis)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
X_train_data.extend(X_train)
y_train_data.extend(y_train)
event_timex_data = (X_test, y_test)
X, y = loo_X_ja_y_main_events(layer, viis)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
X_train_data.extend(X_train)
y_train_data.extend(y_train)
main_event_data = (X_test, y_test)
X, y = loo_X_ja_y_sub_events(layer, viis)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
X_train_data.extend(X_train)
y_train_data.extend(y_train)
sub_event_data = (X_test, y_test)

test_data = [event_dct_data, event_timex_data, main_event_data, sub_event_data]
print(len(X_train_data), len(y_train_data))

4010
4010
559
559
2552
2552
3127
3127
9222 9222


In [ ]:
X_train_data_padded = []
lenghts = set()
for emb in X_train_data:
  emblist = emb.tolist()
  if len(emblist) != 1536:
    korda = 1536 - len(emblist)
    for i in range(korda):
      emblist.append(0.0)
  X_train_data_padded.append(emblist)
  lenghts.add(len(emb))
print(lenghts)
lenghts = set()
for emb in X_train_data_padded:
  lenghts.add(len(emb))
print(lenghts)
#X_train_data = list(X_train_data)
#y_train_data = list(y_train_data)
proportions = get_proportions(y_train_data, osakaal=False)
print("Treenimisel relatsioonitüüpid:", proportions)
all_y = []
for ryhm in test_data:
  all_y.extend(ryhm[1])
proportions = get_proportions(all_y, osakaal=False)
print("Testimisel relatsioonitüüpid:", proportions)

{768, 1536}
{1536}
Treenimisel relatsioonitüüpid: 
BEFORE-OR-OVERLAP: 957
INCLUDES: 1432
VAGUE: 1314
SIMULTANEOUS: 625
IDENTITY: 14
BEFORE: 2453
AFTER: 1113
IS_INCLUDED: 855
OVERLAP-OR-AFTER: 459

Testimisel relatsioonitüüpid: 
INCLUDES: 168
BEFORE-OR-OVERLAP: 102
VAGUE: 147
SIMULTANEOUS: 70
IDENTITY: 1
BEFORE: 268
AFTER: 123
IS_INCLUDED: 95
OVERLAP-OR-AFTER: 52



In [ ]:
print(len(tulemused[0]))
print(tulemused[0].count("IS_INCLUDED"))
print(tulemused[1].count("IS_INCLUDED"))
print(tulemused[0].count("SIMULTANEOUS"))
print(tulemused[1].count("SIMULTANEOUS"))
report = classification_report(tulemused[0], tulemused[1])
print(report)
print("Micro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="micro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="micro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="micro"))
print("Macro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="macro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="macro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="macro"))
print("Weighted")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="weighted"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="weighted"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="weighted"))

print("TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)")
oigeid = 0
for i in range(len(tulemused[0])):
  if tulemused[0][i] == tulemused[1][i]:
    oigeid += 1
print("Skoor:", str(oigeid / len(tulemused[0])))

9222
891
855
585
625
                   precision    recall  f1-score   support

            AFTER       0.54      0.53      0.53      1139
           BEFORE       0.74      0.70      0.72      2578
BEFORE-OR-OVERLAP       0.42      0.45      0.44       888
         IDENTITY       0.00      0.00      0.00         1
         INCLUDES       0.57      0.55      0.56      1468
      IS_INCLUDED       0.53      0.51      0.52       891
 OVERLAP-OR-AFTER       0.14      0.19      0.16       336
     SIMULTANEOUS       0.34      0.36      0.35       585
            VAGUE       0.45      0.44      0.45      1336

         accuracy                           0.54      9222
        macro avg       0.41      0.42      0.41      9222
     weighted avg       0.55      0.54      0.54      9222

Micro
- recall: 0.5373021036651485
- precsision: 0.5373021036651485
- f1-score: 0.5373021036651485
Macro
- recall: 0.41583446193019835
- precsision: 0.4141750407297642
- f1-score: 0.4143673192848268
Weighted
-

In [ ]:
ryhma_nimi = {0:"event_dct_data", 1:"event_timex_data", 2:"main_event_data", 3:"sub_event_data"}
ryhma_nr = 0
for ryhm in test_data:
  print("Rühm:", ryhma_nimi.get(ryhma_nr))

  X_test = []
  lenghts = set()
  for emb in ryhm[0]:
    emblist = emb.tolist()
    if len(emblist) != 1536:
      korda = 1536 - len(emblist)
      for i in range(korda):
        emblist.append(0.0)
    X_test.append(emblist)
    lenghts.add(len(emb))

  y_pred = clf5.predict(X_test)

  report = classification_report(y_pred, ryhm[1])
  print(report)
  print("Micro")
  print("- recall:", recall_score(y_pred, ryhm[1], average="micro"))
  print("- precsision:", precision_score(y_pred, ryhm[1], average="micro"))
  print("- f1-score:", f1_score(y_pred, ryhm[1], average="micro"))
  print("Macro")
  print("- recall:", recall_score(y_pred, ryhm[1], average="macro"))
  print("- precsision:", precision_score(y_pred, ryhm[1], average="macro"))
  print("- f1-score:", f1_score(y_pred, ryhm[1], average="macro"))
  print("Weighted")
  print("- recall:", recall_score(y_pred, ryhm[1], average="weighted"))
  print("- precsision:", precision_score(y_pred, ryhm[1], average="weighted"))
  print("- f1-score:", f1_score(y_pred, ryhm[1], average="weighted"))

  print("TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)")
  oigeid = 0
  for i in range(len(y_pred)):
    if y_pred[i] == ryhm[1][i]:
      oigeid += 1
  print("Skoor:", str(oigeid / len(y_pred)))

  ryhma_nr += 1
  print()

Rühm: event_dct_data
                   precision    recall  f1-score   support

            AFTER       0.48      0.48      0.48        46
           BEFORE       0.86      0.81      0.83       156
BEFORE-OR-OVERLAP       0.56      0.68      0.61        28
         INCLUDES       0.69      0.65      0.67       104
      IS_INCLUDED       0.00      0.00      0.00         0
 OVERLAP-OR-AFTER       0.30      0.60      0.40        10
     SIMULTANEOUS       0.00      0.00      0.00         1
            VAGUE       0.45      0.43      0.44        56

         accuracy                           0.66       401
        macro avg       0.42      0.46      0.43       401
     weighted avg       0.68      0.66      0.67       401

Micro
- recall: 0.6608478802992519
- precsision: 0.6608478802992519
- f1-score: 0.6608478802992519
Macro
- recall: 0.45586777353081703
- precsision: 0.4183507296633465
- f1-score: 0.4299044350842652
Weighted
- recall: 0.6608478802992519
- precsision: 0.68029663761095


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricW

                   precision    recall  f1-score   support

            AFTER       0.00      0.00      0.00         0
           BEFORE       0.00      0.00      0.00         1
BEFORE-OR-OVERLAP       0.50      1.00      0.67         3
         INCLUDES       1.00      1.00      1.00         1
      IS_INCLUDED       0.79      0.64      0.71        36
 OVERLAP-OR-AFTER       0.33      0.50      0.40         2
     SIMULTANEOUS       0.47      0.58      0.52        12
            VAGUE       0.00      0.00      0.00         1

         accuracy                           0.62        56
        macro avg       0.39      0.47      0.41        56
     weighted avg       0.67      0.62      0.63        56

Micro
- recall: 0.625
- precsision: 0.625
- f1-score: 0.625
Macro
- recall: 0.4652777777777778
- precsision: 0.38663793103448274
- f1-score: 0.4116096866096866
Weighted
- recall: 0.625
- precsision: 0.6663998357963876
- f1-score: 0.6339133089133089
TempEval-2 järgi (õigete vastuste arv / 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                   precision    recall  f1-score   support

            AFTER       0.51      0.61      0.56        33
           BEFORE       0.63      0.54      0.58        76
BEFORE-OR-OVERLAP       0.36      0.34      0.35        44
         INCLUDES       0.31      0.46      0.37        24
      IS_INCLUDED       0.37      0.31      0.34        35
 OVERLAP-OR-AFTER       0.21      0.25      0.23        12
     SIMULTANEOUS       0.34      0.39      0.37        28
            VAGUE       0.42      0.38      0.40        61

         accuracy                           0.43       313
        macro avg       0.39      0.41      0.40       313
     weighted avg       0.44      0.43      0.43       313

Micro
- recall: 0.43130990415335463
- precsision: 0.43130990415335463
- f1-score: 0.43130990415335463
Macro
- recall: 0.40987109399803534
- precsision: 0.39364654442779445
- f1-score: 0.39813360940598225
Weighted
- recall: 0.43130990415335463
- precsision: 0.44232591157511286
- f1-score: 

### Top3 mudelid erinevate kombinatsioonidega



```
------------------------ multiword_tüüp: main_event_embedding
------ kiht: bert_embeddings_fixed
###  0.44 0.36 0.45 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.42 0.35 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.39 0.32 0.39 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_add_fixed
###  0.44 0.36 0.44 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.42 0.35 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.38 0.31 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.44 0.36 0.44 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.43 0.35 0.44 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.37 0.31 0.37 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.43 0.35 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.4 0.33 0.41 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.37 0.31 0.37 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.43 0.35 0.44 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.41 0.34 0.42 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.38 0.31 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)

------------------------ multiword_tüüp: event_embeddingute_keskmised
------ kiht: bert_embeddings_fixed
###  0.44 0.37 0.45 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.42 0.34 0.42 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.39 0.33 0.4 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_add_fixed
###  0.44 0.36 0.45 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.43 0.35 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.38 0.31 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.43 0.35 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.43 0.35 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.38 0.31 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.43 0.35 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.4 0.33 0.41 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.36 0.3 0.36 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.45 0.37 0.46 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.43 0.35 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.38 0.32 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)

------------------------ multiword_tüüp: event_embeddingute_kaalutatud_keskmised
------ kiht: bert_embeddings_fixed
###  0.44 0.36 0.45 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.42 0.35 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.4 0.33 0.4 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_add_fixed
###  0.44 0.37 0.45 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.42 0.34 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.37 0.31 0.37 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.44 0.36 0.44 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.43 0.36 0.44 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.37 0.31 0.37 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.43 0.35 0.44 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.4 0.33 0.41 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.36 0.3 0.36 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.45 0.37 0.45 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.42 0.34 0.42 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.38 0.32 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)

------------------------ multiword_tüüp: event_embeddingute_kaalutatud_keskmised_sonaliigid
------ kiht: bert_embeddings_fixed
###  0.44 0.36 0.45 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.41 0.34 0.41 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.39 0.32 0.4 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_add_fixed
###  0.44 0.36 0.44 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.42 0.35 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.37 0.31 0.37 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_all_add_fixed
###  0.43 0.35 0.43 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.43 0.36 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.38 0.31 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_lastlayer_fixed
###  0.43 0.35 0.43 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.4 0.33 0.4 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.36 0.3 0.36 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
------ kiht: bert_embeddings_penultimatelayer_fixed
###  0.45 0.37 0.45 - MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)
###  0.42 0.34 0.42 - SVC(random_state=0, kernel="linear", C=0.025)
###  0.38 0.32 0.38 - LinearSVC(random_state=0, C=0.025, tol=1.0e-6, max_iter=5000, dual=False)
```



In [ ]:
X, y = loo_X_ja_y_main_events("bert_embeddings_add_fixed", 0)
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=4, solver="adam", max_iter=10000, random_state=0))
#clf5 = make_pipeline(StandardScaler(), SVC(random_state=0, kernel="linear", C=0.025, class_weight="balanced"))
tulemused = ristvalideeri(clf5, X, y)
#tulemused = prindi_tulemused(clf5, "MLPClassifier(alpha=0.2, max_iter=5000, random_state=0)", X, y, "bert_embeddings_penultimatelayer_fixed")

## tlink-event-dct lihtsustatud relatsioonitüübid
IDENTITY, INCLUDES, IS_INCLUDED ja SIMULTANEOUS => OVERLAP

In [45]:
def relatsioonityypide_lihtsustamine(y):
  for i in range(len(y)):
    if y[i] in ["IDENTITY", "INCLUDES", "IS_INCLUDED", "SIMULTANEOUS"]:
      y[i] = "OVERLAP"

In [46]:
X, y = loo_X_ja_y_DCT("bert_embeddings_penultimatelayer_fixed", 0, False)
relatsioonityypide_lihtsustamine(y)
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=2.8, solver="lbfgs", max_iter=5000, random_state=0))
tulemused = ristvalideeri(clf5, X, y)

4010
4010
split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9


In [47]:
report = classification_report(tulemused[0], tulemused[1])
print(report)
print("Micro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="micro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="micro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="micro"))
print("Macro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="macro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="macro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="macro"))
print("Weighted")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="weighted"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="weighted"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="weighted"))

print("TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)")
oigeid = 0
for i in range(len(tulemused[0])):
  if tulemused[0][i] == tulemused[1][i]:
    oigeid += 1
print("Skoor:", str(oigeid / len(tulemused[0])))

                   precision    recall  f1-score   support

            AFTER       0.51      0.49      0.50       478
           BEFORE       0.86      0.82      0.84      1597
BEFORE-OR-OVERLAP       0.60      0.73      0.66       313
          OVERLAP       0.67      0.63      0.65       992
 OVERLAP-OR-AFTER       0.13      0.20      0.15       120
            VAGUE       0.45      0.46      0.45       510

         accuracy                           0.66      4010
        macro avg       0.53      0.55      0.54      4010
     weighted avg       0.67      0.66      0.67      4010

Micro
- recall: 0.6603491271820449
- precsision: 0.6603491271820449
- f1-score: 0.6603491271820449
Macro
- recall: 0.5543356227724944
- precsision: 0.5340770998528938
- f1-score: 0.5414562678712995
Weighted
- recall: 0.6603491271820449
- precsision: 0.6749083636422127
- f1-score: 0.6664594327796154
TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)
Skoor: 0.6603491271820449


## tlink-event-timex lihtsustatud relatsioonitüübid
IDENTITY, INCLUDES, IS_INCLUDED ja SIMULTANEOUS => OVERLAP

In [57]:
X, y = loo_X_ja_y_eventtimex("bert_embeddings_add_fixed", 0)
print(get_propotions(y))
relatsioonityypide_lihtsustamine(y)
print("Peale lihtsustamist")
print(get_propotions(y))
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=3, solver="sgd", max_iter=10000, random_state=0))
tulemused = ristvalideeri(clf5, X, y)

split 0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9


In [58]:
report = classification_report(tulemused[0], tulemused[1])
print(report)
print("Micro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="micro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="micro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="micro"))
print("Macro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="macro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="macro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="macro"))
print("Weighted")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="weighted"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="weighted"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="weighted"))

print("TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)")
oigeid = 0
for i in range(len(tulemused[0])):
  if tulemused[0][i] == tulemused[1][i]:
    oigeid += 1
print("Skoor:", str(oigeid / len(tulemused[0])))

                   precision    recall  f1-score   support

            AFTER       0.21      0.60      0.32         5
           BEFORE       0.00      0.00      0.00         0
BEFORE-OR-OVERLAP       0.25      0.36      0.30        44
          OVERLAP       0.93      0.84      0.88       498
 OVERLAP-OR-AFTER       0.15      0.33      0.21        12
            VAGUE       0.00      0.00      0.00         0

         accuracy                           0.79       559
        macro avg       0.26      0.36      0.28       559
     weighted avg       0.85      0.79      0.82       559

Micro
- recall: 0.7889087656529516
- precsision: 0.7889087656529516
- f1-score: 0.7889087656529515
Macro
- recall: 0.3560545211147621
- precsision: 0.258842967640295
- f1-score: 0.28469482680440106
Weighted
- recall: 0.7889087656529516
- precsision: 0.8545780180599757
- f1-score: 0.8173388936618443
TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)
Skoor: 0.7889087656529516


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## tlink-main-event lihtsustatud relatsioonitüübid
IDENTITY, INCLUDES, IS_INCLUDED ja SIMULTANEOUS => OVERLAP

In [61]:
X, y = loo_X_ja_y_main_events("bert_embeddings_add_fixed", 0)
relatsioonityypide_lihtsustamine(y)
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=2.8, solver="lbfgs", max_iter=10000, random_state=0))
tulemused = ristvalideeri(clf5, X, y)

split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9


In [62]:
report = classification_report(tulemused[0], tulemused[1])
print(report)
print("Micro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="micro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="micro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="micro"))
print("Macro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="macro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="macro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="macro"))
print("Weighted")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="weighted"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="weighted"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="weighted"))

print("TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)")
oigeid = 0
for i in range(len(tulemused[0])):
  if tulemused[0][i] == tulemused[1][i]:
    oigeid += 1
print("Skoor:", str(oigeid / len(tulemused[0])))

                   precision    recall  f1-score   support

            AFTER       0.57      0.53      0.55       399
           BEFORE       0.64      0.61      0.62       581
BEFORE-OR-OVERLAP       0.26      0.40      0.32       128
          OVERLAP       0.67      0.59      0.63      1002
 OVERLAP-OR-AFTER       0.22      0.35      0.27        96
            VAGUE       0.55      0.63      0.59       346

         accuracy                           0.57      2552
        macro avg       0.49      0.52      0.50      2552
     weighted avg       0.59      0.57      0.58      2552

Micro
- recall: 0.5713166144200627
- precsision: 0.5713166144200627
- f1-score: 0.5713166144200627
Macro
- recall: 0.5179617433576472
- precsision: 0.4851673540162245
- f1-score: 0.49563632546831804
Weighted
- recall: 0.5713166144200627
- precsision: 0.5928681135886226
- f1-score: 0.5793390353311142
TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)
Skoor: 0.5713166144200627


## tlink-subordinate-events lihtsustatud relatsioonitüübid
IDENTITY, INCLUDES, IS_INCLUDED ja SIMULTANEOUS => OVERLAP

In [53]:
X, y = loo_X_ja_y_sub_events("bert_embeddings_penultimatelayer_fixed", 1)
relatsioonityypide_lihtsustamine(y)
clf5 = make_pipeline(StandardScaler(), MLPClassifier(alpha=1, solver="lbfgs", max_iter=10000, random_state=0))
tulemused = ristvalideeri(clf5, X, y)

split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9


In [54]:
report = classification_report(tulemused[0], tulemused[1])
print(report)
print("Micro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="micro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="micro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="micro"))
print("Macro")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="macro"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="macro"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="macro"))
print("Weighted")
print("- recall:", recall_score(tulemused[0], tulemused[1], average="weighted"))
print("- precsision:", precision_score(tulemused[0], tulemused[1], average="weighted"))
print("- f1-score:", f1_score(tulemused[0], tulemused[1], average="weighted"))

print("TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)")
oigeid = 0
for i in range(len(tulemused[0])):
  if tulemused[0][i] == tulemused[1][i]:
    oigeid += 1
print("Skoor:", str(oigeid / len(tulemused[0])))

                   precision    recall  f1-score   support

            AFTER       0.54      0.56      0.55       377
           BEFORE       0.58      0.54      0.56       689
BEFORE-OR-OVERLAP       0.37      0.40      0.38       382
          OVERLAP       0.56      0.53      0.54      1038
 OVERLAP-OR-AFTER       0.16      0.26      0.20        89
            VAGUE       0.41      0.40      0.40       552

         accuracy                           0.49      3127
        macro avg       0.44      0.45      0.44      3127
     weighted avg       0.50      0.49      0.49      3127

Micro
- recall: 0.4912056283978254
- precsision: 0.4912056283978254
- f1-score: 0.4912056283978254
Macro
- recall: 0.4496777065762109
- precsision: 0.43657342359527523
- f1-score: 0.44103176926006754
Weighted
- recall: 0.4912056283978254
- precsision: 0.49991400969542404
- f1-score: 0.49491001409752017
TempEval-2 järgi (õigete vastuste arv / kogu vastuste arv)
Skoor: 0.4912056283978254
